This notebook contains various convergence tests for MPFA.

They are included mainly to complement the tests for MPSA.

In [1]:
import numpy as np
from scipy.sparse.linalg import spsolve
import scipy.sparse as sps
import sympy
from math import pi

import setup_grids

from porepy.numerics.fv import mpfa, fvutils

from porepy.params.tensor import SecondOrderTensor as perm
from porepy.params import bc

In [2]:
# Analytical solution
x, y = sympy.symbols('x y')
u = x * (1-x) * sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = perm(2, np.ones(g.num_cells))

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros(g.num_faces)
        u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes - div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        flux_num = flux * u_num + bound_flux * u_bound

        # Exact solution
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']


u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(f_cart_nopert,'flux error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'pressure error')
print(f_cart_pert,'flux error')


u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(f_triang_nopert,'flux error')


u_triang_pert, f_triang_pert = run_convergence('triangular', 0.5)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'pressure error')
print(f_triang_pert,'flux error')

# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2] - 0.0065122960444455652) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.011371514390600204) < 1e-10
assert np.abs(u_cart_pert[2] - 0.0073837999427839654) < 1e-10
assert np.abs(f_cart_pert[2] - 0.016113648245047234) < 1e-10

assert np.abs(u_triang_nopert[2] - 0.0036469571622974348) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.0093869826805719359) < 1e-10
assert np.abs(u_triang_pert[2] - 0.0041728065884230805) < 1e-10
assert np.abs(f_triang_pert[2] - 0.021571116081233219) < 1e-10

Cartesian errors - no perturbations
[0.097446236736480271, 0.02563304567729676, 0.0065122960444455669] pressure error
[0.11792041665757692, 0.038488831450398144, 0.011371514390600216] flux error
Cartesian errors - perturbation 0.5
[0.10329022796802324, 0.026622998188774918, 0.007383799942783811] pressure error
[0.10417700714847179, 0.049037576768854028, 0.016113648245047192] flux error
Triangular errors - no perturbations
[0.051690983672416313, 0.01415183595713736, 0.0036469571622975492] pressure error
[0.10250239674096234, 0.032343692758249991, 0.0093869826805719064] flux error
Triangular errors - perturbation 0.5
[0.056804048914338157, 0.015912520716734581, 0.0041728065884231412] pressure error
[0.085677264135833445, 0.040439945237318149, 0.021571116081233101] flux error


# Neumann boundary condition

In [3]:
# Analytical solution
x, y = sympy.symbols('x y')
u = x * (1-x) * sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = perm(2, np.ones(g.num_cells))
        
        # Exact solution
        xf = g.face_centers
        xc = g.cell_centers
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        n = g.nodes
        top = bc.face_on_side(g, 'ymax')
        bot = bc.face_on_side(g, 'ymin')
        left = bc.face_on_side(g, 'xmin')
        right = bc.face_on_side(g, 'xmax')

        dir_faces = np.asarray(right[0])
        neu_faces = np.hstack((left[0], bot[0], top[0]))
        bound_cond = bc.BoundaryCondition(g, dir_faces, ['dir'] * len(dir_faces))
                
        # MPFA discretization, and system matrix
        flux, bound_flux = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        nfi, _, sgn = sps.find(g.cell_faces[neu_faces,:])
        u_bound = np.zeros(g.num_faces)
        u_bound[dir_faces] = u_f(xf[0, dir_faces], xf[1, dir_faces])
        # innflow is always positive, so need to flip flux according to normal direction.
        u_bound[neu_faces[nfi]] = flux_ex[neu_faces[nfi]] * (- sgn)

        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes - div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        flux_num = flux * u_num + bound_flux * u_bound

        # Exact solution
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

## No tests on perturbed grids - I'm too lazy to find faces of the respective boundaries


u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(f_cart_nopert,'flux error')

u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(f_triang_nopert,'flux error')

# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2] - 0.01341819229106214) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.0014594922987503009) < 1e-10

assert np.abs(u_triang_nopert[2] - 0.0097488594883563227) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.0025090432921278465) < 1e-10


Cartesian errors - no perturbations
[0.21776303386171969, 0.053854028644456195, 0.013418192291060563] pressure error
[0.020449627547754923, 0.0055895984138844296, 0.0014594922987497814] flux error
Triangular errors - no perturbations
[0.16013945454985359, 0.039260341126552883, 0.0097488594883491635] pressure error
[0.034067205970900939, 0.0090993177124244796, 0.0025090432921258525] flux error


# Heterogeneous permeability field
Jump in upper right corner

In [4]:
# Analytical solution

kappa_list = [1e-6, 1, 1e6]

### End of parameter definitions

# Permeability tensor, scalar for simplicity

def chi(x, y):
    return np.logical_and(np.greater(x, 0.5), np.greater(y, 0.5))

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(2 * x * pi) * sympy.sin(2 * y * pi) 
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 3
grid_type = 'cart'

def run_convergence(grid_type, pert):
    u_err = np.zeros((num_refs, len(kappa_list)))
    flux_err = np.copy(u_err)
    for iter1, g in enumerate(setup_grids.grid_sequence_fixed_lines(basedim, num_refs, grid_type, pert, subdom_func=chi)):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        for iter2, kappa in enumerate(kappa_list):
            char_func_cells = chi(g.cell_centers[0], g.cell_centers[1]) * 1.
            perm_vec = (1 - char_func_cells) + kappa * char_func_cells
            # Permeability tensor
            k = perm(2, perm_vec)
            bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
            bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
            flux, bound_flux = mpfa.mpfa(g, k, bound_cond)

            xc = g.cell_centers
            xf = g.face_centers
            char_func_bound = chi(xf[0, bound_faces], xf[1, bound_faces]) * 1
            
            u_bound = np.zeros(g.num_faces)
            u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces]) \
                                / ((1 - char_func_bound) + kappa * char_func_bound)

            div = fvutils.scalar_divergence(g)
            a = div * flux

            b = rhs_f(xc[0], xc[1]) * g.cell_volumes - div * bound_flux * u_bound

            u_num = spsolve(a, b)
            u_ex = u_f(xc[0], xc[1]) / ((1 - char_func_cells) + kappa * char_func_cells)
            
            flux_num = flux * u_num + bound_flux * u_bound
            du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
            flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
            flux_diff = flux_num - flux_ex

            u_err[iter1, iter2] = (np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                         np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
            flux_err[iter1, iter2] = (np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                            np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))

    return u_err, flux_err

grids = ['cart', 'triangular']

## No tests on perturbed grids - I'm too lazy to find faces of the respective boundaries

u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(f_cart_nopert,'flux error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'pressure error')
print(f_cart_pert,'flux error')


u_triang_nopert, f_triang_nopert = run_convergence('triangular', 0)
print('Triangular errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(f_triang_nopert,'flux error')


u_triang_pert, f_triang_pert = run_convergence('triangular', 0.5)
print('Triangular errors - perturbation 0.5')
print(u_triang_pert,'pressure error')
print(f_triang_pert,'flux error')

# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2, 0] - 0.01295075) < 1e-7
assert np.abs(f_cart_nopert[2, 0] - 0.00645454) < 1e-7
assert np.abs(u_cart_pert[2, 0] - 0.01484212) < 1e-7
assert np.abs(f_cart_pert[2, 0] - 0.01687373) < 1e-7

assert np.abs(u_triang_nopert[2, 0] - 0.00458976) < 1e-7
assert np.abs(f_triang_nopert[2, 0] - 0.00457494) < 1e-7
assert np.abs(u_triang_pert[2, 0] - 0.00709532) < 1e-7
assert np.abs(f_triang_pert[2, 0] - 0.01712213) < 1e-7


Cartesian errors - no perturbations
[[ 0.23370055  0.23370055  0.23370055]
 [ 0.05302929  0.05302929  0.05302929]
 [ 0.01295075  0.01295075  0.01295075]] pressure error
[[ 0.11072073  0.11072073  0.11072073]
 [ 0.02617215  0.02617215  0.02617215]
 [ 0.00645454  0.00645454  0.00645454]] flux error
Cartesian errors - perturbation 0.5
[[ 0.28144397  0.3116494   0.28115807]
 [ 0.08134885  0.0741203   0.06868011]
 [ 0.01484212  0.01673688  0.01728718]] pressure error
[[ 0.18233422  0.18698603  0.20433553]
 [ 0.05353656  0.05186143  0.05246204]
 [ 0.01687373  0.01673338  0.01674571]] flux error
Triangular errors - no perturbations
[[ 0.04216252  0.08406888  0.05886725]
 [ 0.0160945   0.02320724  0.01833959]
 [ 0.00458976  0.00619313  0.00506888]] pressure error
[[ 0.07128838  0.0699927   0.07409537]
 [ 0.01785551  0.01743713  0.01832159]
 [ 0.00457494  0.00442067  0.00466632]] flux error
Triangular errors - perturbation 0.5
[[ 0.0909746   0.18646187  0.14360106]
 [ 0.0305652   0.03734376  0.

# 3d test cases

In [5]:
# Analytical solution
x, y, z = sympy.symbols('x y z')
u = sympy.cos(x) * sympy.sin(z) * sympy.cosh(y)
u_f = sympy.lambdify((x, y, z), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
duz = sympy.diff(u, z)
dux_f = sympy.lambdify((x, y, z), dux, 'numpy')
duy_f = sympy.lambdify((x, y, z), duy, 'numpy')
duz_f = sympy.lambdify((x, y, z), duz, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y) - sympy.diff(duz, z)
rhs_f = sympy.lambdify((x, y, z), rhs, 'numpy')

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1, 1])
basedim = np.array([base, base, base])
num_refs = 3

def run_convergence(grid_type, pert):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert, domain=domain):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = perm(3, np.ones(g.num_cells))

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux

        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros(g.num_faces)
        u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces], xf[2, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1], xc[2]) * g.cell_volumes - div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = spsolve(a, b)
        flux_num = flux * u_num + bound_flux * u_bound

        # Exact solution
        u_ex = u_f(xc[0], xc[1], xc[2])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1], xf[2]),
                                 duy_f(xf[0], xf[1], xf[2]),
                                 duz_f(xf[0], xf[1], xf[2])))
        flux_ex = -np.sum(g.face_normals * du_ex_faces, axis=0)
        flux_diff = flux_num - flux_ex

        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

u_cart_nopert, f_cart_nopert = run_convergence('cart', 0)
print('Cartesian errors - no perturbations')
print(u_cart_nopert,'pressure error')
print(f_cart_nopert,'flux error')

u_cart_pert, f_cart_pert = run_convergence('cart', 0.5)
print('Cartesian errors - perturbation 0.5')
print(u_cart_pert,'pressure error')
print(f_cart_pert,'flux error')


u_triang_nopert, f_triang_nopert = run_convergence('tetrahedral', 0)
print('Tetrehedral errors - no perturbations')
print(u_triang_nopert,'pressure error')
print(f_triang_nopert,'flux error')

u_triang_pert, f_triang_pert = run_convergence('tetrahedral', 0.3)
print('Tetrahedral errors - perturbation 0.3')
print(u_triang_pert,'pressure error')
print(f_triang_pert,'flux error')

# EK: These values were hard-coded 11.06.2017.
assert np.abs(u_cart_nopert[2] -0.00030564723918128137 ) < 1e-10
assert np.abs(f_cart_nopert[2] - 0.0028725567629342458) < 1e-10
assert np.abs(u_cart_pert[2] - 0.00032732824057441688) < 1e-10
assert np.abs(f_cart_pert[2] - 0.0041613784496996173) < 1e-10

assert np.abs(u_triang_nopert[2] - 0.0002023013247379874) < 1e-10
assert np.abs(f_triang_nopert[2] - 0.0061270858172900334) < 1e-10
assert np.abs(u_triang_pert[2] - 0.00022084341565487475) < 1e-10
assert np.abs(f_triang_pert[2] - 0.0068174626666119582) < 1e-10

Cartesian errors - no perturbations
[0.0038211823411524516, 0.0011419336515988462, 0.00030564723918128137] pressure error
[0.032618619555569772, 0.010014070049802667, 0.0028725567629342458] flux error
Cartesian errors - perturbation 0.5
[0.0041902640162658333, 0.0012081169906913995, 0.00032732824057441688] pressure error
[0.028005576371852938, 0.010688450320353534, 0.0041613784496996173] flux error
Triangular errors - no perturbations
[0.0027818874239716501, 0.00077613734127503903, 0.0002023013247379874] pressure error
[0.029950656631231944, 0.013118747871203309, 0.0061270858172900334] flux error
Triangular errors - perturbation 0.3
[0.0032035289802089984, 0.00083783762868375839, 0.00022084341565487475] pressure error
[0.033346275518853732, 0.015048750763077498, 0.0068174626666119582] flux error
